# 서로 다른 아키텍처

## json 파일 읽어오기

In [1]:
import json
from pathlib import Path
from typing import List, Dict, Union

def load_file(file_path: Union[str, Path]) -> List[str]:

    path = Path(file_path)

    # JSON 파일 읽기
    with path.open('r', encoding='utf-8') as f:
        predictions = json.load(f)
        
    # dictionary 값들을 리스트로 변환
    if not isinstance(predictions, dict):
        raise ValueError("JSON 파일은 dictionary 형태여야 합니다.")
        
    return list(predictions.values())

In [2]:
bert = './resources/checkpoint/extraction/origin/eeeyounglee_kpfbert-korquad-1/predictions.json'
roberta = './resources/checkpoint/extraction/origin/hongzoh_roberta-large-qa-korquad-v1_batch32/predictions.json'
kobigbird = './resources/checkpoint/extraction/origin/YuJungSoo_kobigbird-pure45-19926792/predictions.json'

In [3]:
bert = load_file(bert)
roberta = load_file(roberta)
kobigbird = load_file(kobigbird)

## 답 불러오기

In [13]:
from datasets import load_from_disk
valid = load_from_disk('./resources/data/train_dataset/validation/')

In [14]:
import pandas as pd

valid_df = pd.DataFrame(valid)

In [15]:
question_answers_text_df = valid_df[['question', 'answers']].copy()
question_answers_text_df['answers_text'] = question_answers_text_df['answers'].apply(lambda x: x['text'])
answers = question_answers_text_df[['answers_text']]

In [16]:
answers = [ans[0] for ans in answers['answers_text']]

In [17]:
import numpy as np
import pandas as pd
from typing import List, Dict

def calculate_error_correlations(predictions_dict: Dict[str, List[str]], 
                               answers: List[str],
                               save_path: str = None) -> pd.DataFrame:
    """
    여러 모델의 predictions와 정답을 비교하여 에러 상관계수를 계산하고 시각화합니다.
    
    Args:
        predictions_dict: Dict[str, List[str]] - 각 모델별 예측값 딕셔너리
            - key: 모델 이름
            - value: 예측값 리스트
        answers: List[str] - 정답 리스트
        save_path: str - 히트맵 저장 경로 (선택사항)
    
    Returns:
        pd.DataFrame - 에러 상관계수 행렬
    """
    # 각 모델의 에러 패턴을 저장할 딕셔너리 (1: 오답, 0: 정답)
    error_patterns = {}
    
    # 각 모델별 에러 패턴 계산
    for model_name, preds in predictions_dict.items():
        errors = []
        for pred, ans in zip(preds, answers):
            # EM 기준으로 정답 여부 판단 (정답:0, 오답:1)
            is_error = 0 if pred == ans else 1
            errors.append(is_error)
        error_patterns[model_name] = errors
    
    # 에러 패턴을 DataFrame으로 변환
    error_df = pd.DataFrame(error_patterns)
    
    # 에러 상관계수 계산
    correlation_matrix = error_df.corr()
    
    return correlation_matrix

def analyze_model_pairs(correlation_matrix: pd.DataFrame, 
                       threshold: float = 0.5) -> List[tuple]:
    """
    상관계수가 높은 모델 쌍을 찾아 반환합니다.
    
    Args:
        correlation_matrix: pd.DataFrame - 에러 상관계수 행렬
        threshold: float - 높은 상관관계로 판단할 기준값 (기본값: 0.5)
    
    Returns:
        List[tuple] - (model1, model2, correlation) 형태의 튜플 리스트
    """
    high_correlations = []
    
    # 상삼각 행렬만 검사 (대각선 제외)
    for i in range(len(correlation_matrix.index)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            correlation = correlation_matrix.iloc[i, j]
            if abs(correlation) >= threshold:
                model1 = correlation_matrix.index[i]
                model2 = correlation_matrix.columns[j]
                high_correlations.append((model1, model2, correlation))
    
    # 상관계수 절대값 기준으로 정렬
    high_correlations.sort(key=lambda x: abs(x[2]), reverse=True)
    return high_correlations

# 사용 예시
if __name__ == "__main__":
    # 예시 데이터
    predictions_dict = {
        "BERT": bert,
        "RoBERTa": roberta,
        "KoBigBird": kobigbird
    }
    answers = answers
    
    # 상관계수 계산 및 시각화
    correlation_matrix = calculate_error_correlations(
        predictions_dict, 
        answers,
    )
    
    # 높은 상관관계를 가진 모델 쌍 분석
    high_correlations = analyze_model_pairs(correlation_matrix, threshold=0.5)
    
    # 결과 출력
    for model1, model2, corr in high_correlations:
        print(f"{model1} - {model2}: {corr:.3f}")
        if corr > 0.7:
            print(f"  ⚠️ {model1}과 {model2} 중 하나만 선택하는 것을 권장합니다.")

BERT - RoBERTa: 0.544
RoBERTa - KoBigBird: 0.528
BERT - KoBigBird: 0.515


# 앙상블 모델 비교

In [18]:
first = load_file('./correlation/bert_cnn.json')
second = load_file('./correlation/kobigbird_sd.json')
third = load_file('./correlation/roberta_hz.json')

In [20]:
predictions_dict = {
    "first": first,
    "second": second,
    "third": third
}
answers = answers

# 상관계수 계산
correlation_matrix = calculate_error_correlations(predictions_dict, answers)

# 높은 상관관계를 가진 모델 쌍 분석
high_correlations = analyze_model_pairs(correlation_matrix, threshold=0)

# 결과 출력
for model1, model2, corr in high_correlations:
    print(f"{model1} - {model2}: {corr:.3f}")
    if corr > 0.7:
        print(f"  ⚠️ {model1}과 {model2} 중 하나만 선택하는 것을 권장합니다.")

first - third: 0.674
first - second: 0.488
second - third: 0.452


## Roberta

In [86]:
first = './resources/checkpoint/extraction/origin/hongzoh_roberta-large-qa-korquad-v1_batch32/predictions.json'
#second = './resources/checkpoint/extraction/origin/hongzoh_roberta-large-qa-korquad-v2_batch32/predictions.json'
second = './resources/checkpoint/extraction/origin/line1029_korquad-finetuned-roberta-large/predictions.json'
third = './resources/checkpoint/extraction/origin/nlpotato_roberta_large_origin_added_korquad/predictions.json'
first = load_file(first)
second = load_file(second)
third = load_file(third)

In [87]:
predictions_dict = {
    "first": first,
    "second": second,
    "third": third
}
answers = answers

# 상관계수 계산
correlation_matrix = calculate_error_correlations(predictions_dict, answers)

# 높은 상관관계를 가진 모델 쌍 분석
high_correlations = analyze_model_pairs(correlation_matrix, threshold=0.5)

# 결과 출력
for model1, model2, corr in high_correlations:
    print(f"{model1} - {model2}: {corr:.3f}")
    if corr > 0.7:
        print(f"  ⚠️ {model1}과 {model2} 중 하나만 선택하는 것을 권장합니다.")

first - third: 0.592
second - third: 0.557
first - second: 0.528


## BERT

In [71]:
first = './resources//predictions.json'
second = './resources/checkpoint/extraction/origin/eeeyounglee_kpfbert-korquad-1/predictions.json'
first = load_file(first)
second = load_file(second)

In [72]:
predictions_dict = {
    "first": first,
    "second": second,
}
answers = answers

# 상관계수 계산
correlation_matrix = calculate_error_correlations(predictions_dict, answers)

# 높은 상관관계를 가진 모델 쌍 분석
high_correlations = analyze_model_pairs(correlation_matrix, threshold=0.5)

# 결과 출력
for model1, model2, corr in high_correlations:
    print(f"{model1} - {model2}: {corr:.3f}")
    if corr > 0.7:
        print(f"  ⚠️ {model1}과 {model2} 중 하나만 선택하는 것을 권장합니다.")

first - second: 0.565
